# 🔬 Explainable AI (XAI): Interpretability & Fairness

**Phase 6: Understanding HOW and WHY ML Models Make Decisions**

---

## 🎯 The Black Box Problem

### До сих пор мы фокусировались на точности:

- ✅ **Phase 1-2:** Accuracy, AUC, F1-Score
- ✅ **Phase 3:** Ensemble methods для улучшения метрик
- ✅ **Phase 4:** Transformers для комплексных паттернов
- ✅ **Phase 5:** Unsupervised learning для anomaly detection

**Но в реальном мире этого недостаточно:**

### 🏥 Медицина
```
Модель: "У пациента рак легких с вероятностью 85%"
Врач: "Почему? На основании каких признаков?"
Модель: "🤷 (black box)"
```
❌ **НЕПРИЕМЛЕМО** - FDA и GDPR требуют объяснимости

### 💳 Финансы
```
Модель: "Кредит отклонен"
Клиент: "Почему? Что нужно улучшить?"
Банк: "🤷 (black box)"
```
❌ **ILLEGAL** - законодательство требует объяснений (Equal Credit Opportunity Act)

### 🏢 Рекрутинг
```
Модель: 95% мужчин получают job offers для tech позиций
HR: "Модель дискриминирует по полу?"
Data Scientist: "🤷 Accuracy 92%, что не так?"
```
❌ **BIAS PROBLEM** - высокая точность != fair predictions

---

## 🚀 Enter Explainable AI (XAI)

### Ключевые вопросы:

1. **Global Interpretability:** Как модель работает в целом?
   - Какие признаки самые важные?
   - Как признаки влияют на predictions?
   - Есть ли нелинейные взаимодействия?

2. **Local Interpretability:** Почему именно это prediction?
   - Почему этому клиенту отказали в кредите?
   - Какие факторы повлияли на этот диагноз?
   - Что нужно изменить для другого результата?

3. **Fairness:** Модель справедлива?
   - Есть ли bias по полу, расе, возрасту?
   - Предсказания калиброваны для всех групп?
   - Demographic parity vs Equal opportunity?

---

## 📊 XAI Methods Overview

### 1. Model-Agnostic Methods (работают с любой моделью)

#### **SHAP (SHapley Additive exPlanations)**
- ✅ **Теоретически обоснован** (game theory, Shapley values)
- ✅ **Consistent:** если модель больше полагается на признак, SHAP value выше
- ✅ **Local + Global interpretability**
- ⚠️ **Computational cost:** TreeSHAP быстр, KernelSHAP медленнее

**Variants:**
- `TreeSHAP`: для tree-based моделей (XGBoost, RandomForest) - очень быстро
- `KernelSHAP`: для любых моделей - медленнее
- `DeepSHAP`: для нейронных сетей

#### **LIME (Local Interpretable Model-agnostic Explanations)**
- ✅ **Fast:** быстрее SHAP для локальных объяснений
- ✅ **Intuitive:** аппроксимирует модель локально простой моделью
- ⚠️ **Unstable:** результаты могут варьироваться
- ⚠️ **Only local:** не даёт глобальной картины

#### **Partial Dependence Plots (PDP)**
- ✅ **Global view:** влияние признака на predictions в среднем
- ✅ **Easy to interpret:** визуально понятно
- ⚠️ **Assumes independence:** может вводить в заблуждение при корреляциях

#### **Permutation Importance**
- ✅ **Simple:** shuffle признак → measure drop in accuracy
- ✅ **True importance:** учитывает корреляции (в отличие от Gini importance)
- ⚠️ **Computational cost:** требует многих перестановок

---

### 2. Model-Specific Methods

#### **Feature Importance (Tree-based models)**
- ✅ **Built-in:** быстро, доступно из коробки
- ⚠️ **Gini bias:** переоценивает high-cardinality features

#### **Attention Weights (Transformers)**
- ✅ **Direct:** модель "говорит", на что смотрит
- ⚠️ **Interpretation caveats:** attention ≠ importance (спорный момент)

#### **Linear Model Coefficients**
- ✅ **Direct interpretation:** вес = влияние признака
- ⚠️ **Only linear models:** не работает для deep learning

---

## 📊 Что мы реализуем

### Dataset: Income Prediction (Adult Census)

**Почему этот датасет?**
- ✅ **Fairness concerns:** пол, раса, возраст могут создавать bias
- ✅ **Real-world problem:** income prediction важен для кредитного скоринга
- ✅ **Interpretability важна:** объяснение, почему кто-то в high/low income группе
- ✅ **Multiple feature types:** numerical, categorical

### Задачи:

**Part 1: Setup & Model Training**
1. Загрузка Adult Census dataset
2. Preprocessing
3. Обучение нескольких моделей (Logistic Regression, RandomForest, XGBoost)

**Part 2: SHAP Analysis**
1. TreeSHAP для RandomForest и XGBoost
2. Global feature importance (summary plots)
3. Local explanations (waterfall plots, force plots)
4. Dependence plots (feature interactions)

**Part 3: LIME Analysis**
1. Local explanations для отдельных predictions
2. Сравнение SHAP vs LIME

**Part 4: Global Interpretability**
1. Partial Dependence Plots (PDP)
2. Individual Conditional Expectation (ICE) curves
3. Permutation Importance

**Part 5: Fairness Analysis**
1. Demographic Parity по полу
2. Equal Opportunity analysis
3. Calibration по группам
4. Bias mitigation strategies

**Part 6: Decision Tree Visualization**
1. Визуализация правил Decision Tree
2. Rule extraction

---


## 💻 Часть 1: Setup и Dataset

### 1.1 Импорт библиотек

In [ ]:
# Базовые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Sklearn - Models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

# Sklearn - Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    roc_curve, precision_recall_curve
)

# Sklearn - Interpretability
from sklearn.inspection import (
    permutation_importance,
    PartialDependenceDisplay,
    partial_dependence
)
from sklearn.tree import plot_tree, export_text

# SHAP
import shap
shap.initjs()  # для визуализации в Jupyter

# LIME
import lime
import lime.lime_tabular

# Настройки визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Reproducibility
np.random.seed(42)

print("\n✅ Все библиотеки загружены")
print(f"SHAP version: {shap.__version__}")
print(f"LIME version: {lime.__version__}")


### 1.2 Загрузка Adult Census Dataset

**Adult Income Dataset** (также известен как Census Income):
- **Задача:** Предсказать, зарабатывает ли человек >50K в год
- **Размер:** ~48,000 записей
- **Признаки:** age, workclass, education, marital-status, occupation, relationship, race, sex, capital-gain, capital-loss, hours-per-week, native-country
- **Target:** income (<=50K or >50K)

**Важно для XAI:**
- Sensitive attributes: sex, race → fairness analysis
- Categorical features → интерпретация категорий
- Real-world implications → ethical AI

In [ ]:
# Загружаем Adult Census dataset
# Используем встроенный dataset или загружаем из UCI repository

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

# Загружаем данные
try:
    df = pd.read_csv(url, names=column_names, sep=',\s*', engine='python', na_values='?')
    print("✅ Dataset загружен из UCI repository")
except:
    print("⚠️ Не удалось загрузить из UCI, создаю синтетический dataset")
    # Создаём синтетический dataset, если нет интернета
    np.random.seed(42)
    n_samples = 30000
    
    df = pd.DataFrame({
        'age': np.random.randint(17, 90, n_samples),
        'workclass': np.random.choice(['Private', 'Self-emp', 'Govt', 'Without-pay'], n_samples, p=[0.7, 0.15, 0.1, 0.05]),
        'education': np.random.choice(['HS-grad', 'Some-college', 'Bachelors', 'Masters', 'Doctorate'], n_samples, p=[0.3, 0.25, 0.3, 0.1, 0.05]),
        'education-num': np.random.randint(1, 16, n_samples),
        'marital-status': np.random.choice(['Married', 'Never-married', 'Divorced'], n_samples, p=[0.5, 0.35, 0.15]),
        'occupation': np.random.choice(['Tech', 'Sales', 'Service', 'Craft', 'Prof'], n_samples, p=[0.15, 0.25, 0.25, 0.2, 0.15]),
        'relationship': np.random.choice(['Husband', 'Wife', 'Own-child', 'Not-in-family'], n_samples, p=[0.3, 0.25, 0.2, 0.25]),
        'race': np.random.choice(['White', 'Black', 'Asian', 'Other'], n_samples, p=[0.8, 0.1, 0.05, 0.05]),
        'sex': np.random.choice(['Male', 'Female'], n_samples, p=[0.67, 0.33]),
        'capital-gain': np.random.choice([0] * 90 + list(np.random.randint(1000, 100000, 10)), n_samples),
        'capital-loss': np.random.choice([0] * 95 + list(np.random.randint(100, 5000, 5)), n_samples),
        'hours-per-week': np.random.randint(1, 100, n_samples),
        'native-country': np.random.choice(['United-States', 'Other'], n_samples, p=[0.9, 0.1]),
    })
    
    # Создаём target с некоторой логикой
    income_prob = (
        (df['age'] > 30).astype(int) * 0.2 +
        (df['education-num'] > 12).astype(int) * 0.3 +
        (df['hours-per-week'] > 40).astype(int) * 0.2 +
        (df['capital-gain'] > 0).astype(int) * 0.25
    ) / 1.0
    
    df['income'] = (np.random.random(n_samples) < income_prob).astype(int)
    df['income'] = df['income'].map({0: '<=50K', 1: '>50K'})

# Убираем пробелы из категориальных признаков
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Удаляем строки с пропусками
df = df.dropna()

print(f"\nРазмер датасета: {df.shape}")
print(f"Признаков: {df.shape[1] - 1}")
print(f"\nПервые строки:")
df.head()


### 1.3 Exploratory Data Analysis (EDA)

Проверим распределение признаков и target variable, особенно для **fairness analysis**.

In [ ]:
# Базовая статистика
print("=" * 60)
print("ИНФОРМАЦИЯ О ДАТАСЕТЕ")
print("=" * 60)
print(df.info())
print("\n" + "=" * 60)
print("СТАТИСТИКА ЧИСЛОВЫХ ПРИЗНАКОВ")
print("=" * 60)
print(df.describe())

# Распределение target
print("\n" + "=" * 60)
print("РАСПРЕДЕЛЕНИЕ INCOME")
print("=" * 60)
print(df['income'].value_counts())
print(f"\nClass balance: {df['income'].value_counts(normalize=True).to_dict()}")


In [ ]:
# Визуализация распределений для fairness analysis
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Income by Sex (FAIRNESS CONCERN!)
pd.crosstab(df['sex'], df['income'], normalize='index').plot(
    kind='bar', ax=axes[0, 0], color=['#e74c3c', '#2ecc71']
)
axes[0, 0].set_title('Income Distribution by Sex', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Sex')
axes[0, 0].set_ylabel('Proportion')
axes[0, 0].legend(title='Income')
axes[0, 0].tick_params(axis='x', rotation=0)

# Income by Race (FAIRNESS CONCERN!)
pd.crosstab(df['race'], df['income'], normalize='index').plot(
    kind='bar', ax=axes[0, 1], color=['#e74c3c', '#2ecc71']
)
axes[0, 1].set_title('Income Distribution by Race', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Race')
axes[0, 1].set_ylabel('Proportion')
axes[0, 1].legend(title='Income')

# Income by Education
pd.crosstab(df['education'], df['income'], normalize='index').plot(
    kind='bar', ax=axes[0, 2], color=['#e74c3c', '#2ecc71']
)
axes[0, 2].set_title('Income Distribution by Education', fontsize=14, fontweight='bold')
axes[0, 2].set_xlabel('Education')
axes[0, 2].set_ylabel('Proportion')
axes[0, 2].legend(title='Income')

# Age distribution
df[df['income'] == '<=50K']['age'].hist(bins=30, alpha=0.5, label='<=50K', ax=axes[1, 0], color='#e74c3c')
df[df['income'] == '>50K']['age'].hist(bins=30, alpha=0.5, label='>50K', ax=axes[1, 0], color='#2ecc71')
axes[1, 0].set_title('Age Distribution by Income', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Age')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].legend()

# Hours per week
df[df['income'] == '<=50K']['hours-per-week'].hist(bins=30, alpha=0.5, label='<=50K', ax=axes[1, 1], color='#e74c3c')
df[df['income'] == '>50K']['hours-per-week'].hist(bins=30, alpha=0.5, label='>50K', ax=axes[1, 1], color='#2ecc71')
axes[1, 1].set_title('Hours per Week by Income', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Hours per Week')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()

# Education years
df.boxplot(column='education-num', by='income', ax=axes[1, 2])
axes[1, 2].set_title('Education Years by Income', fontsize=14, fontweight='bold')
axes[1, 2].set_xlabel('Income')
axes[1, 2].set_ylabel('Education Years')
plt.sca(axes[1, 2])
plt.xticks(rotation=0)

plt.suptitle('EDA: Income Distribution Analysis', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n⚠️ FAIRNESS CONCERNS DETECTED:")
print("- Income distribution varies by SEX")
print("- Income distribution varies by RACE")
print("→ Нужен fairness analysis для проверки модели на bias!")


### 1.4 Preprocessing

**Задачи:**
1. Encoding категориальных признаков
2. Scaling числовых признаков
3. Train/Test split
4. **Важно:** Сохраним sensitive attributes (sex, race) отдельно для fairness analysis

In [ ]:
# Создаём копию для preprocessing
df_processed = df.copy()

# Бинарный target
df_processed['income'] = (df_processed['income'] == '>50K').astype(int)

# Сохраняем sensitive attributes ДО encoding (для fairness analysis)
sensitive_features = df_processed[['sex', 'race']].copy()

# Определяем типы признаков
categorical_cols = [
    'workclass', 'education', 'marital-status', 'occupation',
    'relationship', 'race', 'sex', 'native-country'
]

numerical_cols = [
    'age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'
]

# Label Encoding для категориальных признаков
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col])
    label_encoders[col] = le

print("✅ Label Encoding completed")
print(f"Категориальных признаков: {len(categorical_cols)}")
print(f"Числовых признаков: {len(numerical_cols)}")

# Feature matrix и target
feature_cols = categorical_cols + numerical_cols
X = df_processed[feature_cols]
y = df_processed['income']

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Positive class ratio: {y.mean():.3f}")


In [ ]:
# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Также split для sensitive features (нужно для fairness analysis)
sensitive_train, sensitive_test = train_test_split(
    sensitive_features, test_size=0.2, random_state=42, stratify=y
)

# Scaling только для числовых признаков
scaler = StandardScaler()

# Важно: scaling только на train, затем transform на test
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test_scaled[numerical_cols] = scaler.transform(X_test[numerical_cols])

print("✅ Train/Test split completed")
print(f"Train size: {X_train.shape[0]} ({X_train.shape[0] / len(X):.1%})")
print(f"Test size: {X_test.shape[0]} ({X_test.shape[0] / len(X):.1%})")
print(f"\nTrain positive class: {y_train.mean():.3f}")
print(f"Test positive class: {y_test.mean():.3f}")


### 1.5 Model Training

Обучим 3 модели разной complexity для сравнения interpretability:

1. **Logistic Regression** - простая, линейная, interpretable из коробки
2. **Random Forest** - ensemble, нелинейная, средней сложности
3. **XGBoost** - gradient boosting, высокая точность, сложная

Затем применим XAI методы ко всем трём.

In [ ]:
# Dictionary для хранения моделей и результатов
models = {}
results = {}

print("=" * 70)
print("ОБУЧЕНИЕ МОДЕЛЕЙ")
print("=" * 70)

# 1. Logistic Regression
print("\n[1/3] Training Logistic Regression...")
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train)
lr_pred = lr.predict(X_test_scaled)
lr_pred_proba = lr.predict_proba(X_test_scaled)[:, 1]

models['Logistic Regression'] = lr
results['Logistic Regression'] = {
    'predictions': lr_pred,
    'probabilities': lr_pred_proba,
    'accuracy': accuracy_score(y_test, lr_pred),
    'precision': precision_score(y_test, lr_pred),
    'recall': recall_score(y_test, lr_pred),
    'f1': f1_score(y_test, lr_pred),
    'roc_auc': roc_auc_score(y_test, lr_pred_proba)
}
print("✅ Logistic Regression trained")

# 2. Random Forest
print("\n[2/3] Training Random Forest...")
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_scaled, y_train)
rf_pred = rf.predict(X_test_scaled)
rf_pred_proba = rf.predict_proba(X_test_scaled)[:, 1]

models['Random Forest'] = rf
results['Random Forest'] = {
    'predictions': rf_pred,
    'probabilities': rf_pred_proba,
    'accuracy': accuracy_score(y_test, rf_pred),
    'precision': precision_score(y_test, rf_pred),
    'recall': recall_score(y_test, rf_pred),
    'f1': f1_score(y_test, rf_pred),
    'roc_auc': roc_auc_score(y_test, rf_pred_proba)
}
print("✅ Random Forest trained")

# 3. XGBoost
print("\n[3/3] Training XGBoost...")
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1,
    eval_metric='logloss'
)
xgb_model.fit(X_train_scaled, y_train)
xgb_pred = xgb_model.predict(X_test_scaled)
xgb_pred_proba = xgb_model.predict_proba(X_test_scaled)[:, 1]

models['XGBoost'] = xgb_model
results['XGBoost'] = {
    'predictions': xgb_pred,
    'probabilities': xgb_pred_proba,
    'accuracy': accuracy_score(y_test, xgb_pred),
    'precision': precision_score(y_test, xgb_pred),
    'recall': recall_score(y_test, xgb_pred),
    'f1': f1_score(y_test, xgb_pred),
    'roc_auc': roc_auc_score(y_test, xgb_pred_proba)
}
print("✅ XGBoost trained")

print("\n" + "=" * 70)
print("✅ ВСЕ МОДЕЛИ ОБУЧЕНЫ")
print("=" * 70)


In [ ]:
# Сравнение моделей
comparison_df = pd.DataFrame({
    model_name: {
        'Accuracy': metrics['accuracy'],
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
        'F1-Score': metrics['f1'],
        'ROC AUC': metrics['roc_auc']
    }
    for model_name, metrics in results.items()
}).T

print("\n" + "=" * 70)
print("MODEL COMPARISON")
print("=" * 70)
print(comparison_df.round(4))

# Визуализация сравнения
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Metrics comparison
comparison_df.plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Model')
axes[0].set_ylabel('Score')
axes[0].legend(title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].set_ylim([0, 1])
axes[0].tick_params(axis='x', rotation=0)
axes[0].grid(axis='y', alpha=0.3)

# ROC Curves
for model_name, metrics in results.items():
    fpr, tpr, _ = roc_curve(y_test, metrics['probabilities'])
    auc = metrics['roc_auc']
    axes[1].plot(fpr, tpr, label=f'{model_name} (AUC={auc:.3f})', linewidth=2)

axes[1].plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1)
axes[1].set_title('ROC Curves', fontsize=14, fontweight='bold')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].legend(loc='lower right')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 Наблюдения:")
print("- XGBoost показывает лучшую performance (обычно)")
print("- Random Forest близок к XGBoost")
print("- Logistic Regression проще, но менее точная")
print("\n→ Теперь применим XAI методы для интерпретации!")


---

## 💡 Часть 2: SHAP Analysis

### Что такое SHAP?

**SHAP (SHapley Additive exPlanations)** - unified framework для объяснения predictions любых ML моделей.

**Основан на:**
- **Shapley Values** из теории игр (Lloyd Shapley, Nobel Prize 2012)
- **Идея:** Сколько каждый признак "вкладывает" в предсказание?

**Почему SHAP лучше других методов:**

1. **Теоретически обоснован** - единственный метод с гарантиями:
   - ✅ **Local accuracy:** sum(SHAP values) = prediction - baseline
   - ✅ **Consistency:** если признак важнее в модели B, его SHAP value выше
   - ✅ **Missingness:** отсутствующий признак имеет SHAP = 0

2. **Универсальность:**
   - `TreeSHAP`: для древовидных моделей (RF, XGBoost, LightGBM) - **очень быстро**
   - `KernelSHAP`: для любых моделей (черный ящик) - медленнее
   - `DeepSHAP`: для нейронных сетей
   - `LinearSHAP`: для линейных моделей

3. **Global + Local:**
   - Global importance: среднее |SHAP value| по всем samples
   - Local explanation: SHAP values для конкретного prediction

**Интерпретация SHAP value:**
- `SHAP value > 0`: признак увеличивает prediction (пушит к классу 1)
- `SHAP value < 0`: признак уменьшает prediction (пушит к классу 0)
- `|SHAP value|` = magnitude of effect

---


### 2.1 SHAP для Random Forest

Используем **TreeSHAP** - точный и быстрый алгоритм для древовидных моделей.

In [ ]:
print("=" * 70)
print("SHAP ANALYSIS: RANDOM FOREST")
print("=" * 70)

# TreeExplainer для Random Forest
print("\nCreating TreeExplainer for Random Forest...")
rf_explainer = shap.TreeExplainer(models['Random Forest'])

# Вычисляем SHAP values для test set
# Для скорости используем первые 1000 samples
print("Computing SHAP values (это может занять минуту)...")
rf_shap_values = rf_explainer.shap_values(X_test_scaled.iloc[:1000])

# SHAP возвращает [values_class_0, values_class_1] для бинарной классификации
# Нас интересует класс 1 (>50K)
if isinstance(rf_shap_values, list):
    rf_shap_values_class1 = rf_shap_values[1]
else:
    rf_shap_values_class1 = rf_shap_values

print(f"✅ SHAP values computed")
print(f"Shape: {rf_shap_values_class1.shape}")
print(f"(samples={rf_shap_values_class1.shape[0]}, features={rf_shap_values_class1.shape[1]})")


#### 2.1.1 Global Feature Importance (Summary Plot)

**Summary plot** показывает:
- Какие признаки самые важные (сверху)
- Как значения признака влияют на prediction (цвет: red=high, blue=low)
- Magnitude of effect (SHAP value по оси X)

In [ ]:
# Summary plot - глобальная важность признаков
plt.figure(figsize=(12, 8))
shap.summary_plot(
    rf_shap_values_class1,
    X_test_scaled.iloc[:1000],
    feature_names=feature_cols,
    show=False
)
plt.title('SHAP Summary Plot: Random Forest (Class >50K)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n📊 Интерпретация Summary Plot:")
print("- Features sorted by importance (top = most important)")
print("- Color: red = high feature value, blue = low feature value")
print("- X-axis: SHAP value (impact on prediction)")
print("- Positive SHAP → increases probability of >50K")
print("- Negative SHAP → decreases probability of >50K")
print("\nПримеры:")
print("- capital-gain: high values (red) → positive SHAP → higher income probability")
print("- age: older age (red) → positive SHAP → higher income probability")
print("- education-num: more education (red) → positive SHAP → higher income probability")


In [ ]:
# Bar plot - средняя важность признаков
plt.figure(figsize=(10, 6))
shap.summary_plot(
    rf_shap_values_class1,
    X_test_scaled.iloc[:1000],
    feature_names=feature_cols,
    plot_type='bar',
    show=False
)
plt.title('SHAP Feature Importance: Random Forest', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🎯 Топ-5 самых важных признаков:")
mean_abs_shap = np.abs(rf_shap_values_class1).mean(axis=0)
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Mean |SHAP|': mean_abs_shap
}).sort_values('Mean |SHAP|', ascending=False)

print(feature_importance.head(10).to_string(index=False))


#### 2.1.2 Local Explanations (Waterfall Plot)

**Waterfall plot** объясняет **одно конкретное prediction**:
- Начинаем с baseline (среднее prediction по всему датасету)
- Каждый признак двигает prediction вверх или вниз
- Конечное значение = actual prediction

In [ ]:
# Выберем несколько интересных samples для объяснения
# Sample 1: High income prediction
high_income_idx = np.where((y_test.iloc[:1000] == 1) & (results['Random Forest']['predictions'][:1000] == 1))[0][0]

# Sample 2: Low income prediction
low_income_idx = np.where((y_test.iloc[:1000] == 0) & (results['Random Forest']['predictions'][:1000] == 0))[0][0]

print(f"Selected samples:")
print(f"- High income prediction: index {high_income_idx}")
print(f"- Low income prediction: index {low_income_idx}")

# Waterfall plot для high income
print("\n" + "=" * 70)
print("EXAMPLE 1: HIGH INCOME PREDICTION")
print("=" * 70)

shap.plots.waterfall(
    shap.Explanation(
        values=rf_shap_values_class1[high_income_idx],
        base_values=rf_explainer.expected_value[1] if isinstance(rf_explainer.expected_value, list) else rf_explainer.expected_value,
        data=X_test_scaled.iloc[high_income_idx].values,
        feature_names=feature_cols
    ),
    show=False
)
plt.title(f'Waterfall Plot: Sample {high_income_idx} (Predicted >50K)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nАктуальные значения признаков:")
sample_df = pd.DataFrame({
    'Feature': feature_cols,
    'Value': X_test_scaled.iloc[high_income_idx].values,
    'SHAP': rf_shap_values_class1[high_income_idx]
}).sort_values('SHAP', key=abs, ascending=False)
print(sample_df.head(8).to_string(index=False))


In [ ]:
# Waterfall plot для low income
print("\n" + "=" * 70)
print("EXAMPLE 2: LOW INCOME PREDICTION")
print("=" * 70)

shap.plots.waterfall(
    shap.Explanation(
        values=rf_shap_values_class1[low_income_idx],
        base_values=rf_explainer.expected_value[1] if isinstance(rf_explainer.expected_value, list) else rf_explainer.expected_value,
        data=X_test_scaled.iloc[low_income_idx].values,
        feature_names=feature_cols
    ),
    show=False
)
plt.title(f'Waterfall Plot: Sample {low_income_idx} (Predicted <=50K)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nАктуальные значения признаков:")
sample_df = pd.DataFrame({
    'Feature': feature_cols,
    'Value': X_test_scaled.iloc[low_income_idx].values,
    'SHAP': rf_shap_values_class1[low_income_idx]
}).sort_values('SHAP', key=abs, ascending=False)
print(sample_df.head(8).to_string(index=False))

print("\n💡 Ключевое преимущество SHAP:")
print("Мы можем объяснить КАЖДОЕ prediction, а не только модель в целом!")
print("Это критически важно для:")
print("- Медицинской диагностики (объяснить пациенту)")
print("- Кредитного скоринга (объяснить отказ)")
print("- Fraud detection (почему транзакция подозрительна)")


#### 2.1.3 Dependence Plots (Feature Interactions)

**Dependence plot** показывает, как SHAP value признака зависит от его значения:
- X-axis: значение признака
- Y-axis: SHAP value (impact)
- Color: взаимодействие с другим признаком (автоматически выбирается самое сильное)

Помогает найти **нелинейные эффекты и interactions**.

In [ ]:
# Dependence plots для топ признаков
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

top_features = feature_importance.head(6)['Feature'].values

for idx, feature in enumerate(top_features):
    plt.sca(axes[idx])
    shap.dependence_plot(
        feature,
        rf_shap_values_class1,
        X_test_scaled.iloc[:1000],
        feature_names=feature_cols,
        show=False,
        ax=axes[idx]
    )
    axes[idx].set_title(f'Dependence: {feature}', fontsize=12, fontweight='bold')

plt.suptitle('SHAP Dependence Plots: Feature Interactions', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n📊 Что мы видим:")
print("- Нелинейные зависимости (не просто линейные тренды)")
print("- Feature interactions (цвет показывает взаимодействие с другим признаком)")
print("- Thresholds и breakpoints в влиянии признака")
print("\nПример:")
print("- age: молодой возраст → negative SHAP, пожилой → positive SHAP")
print("- capital-gain: 0 → negative, >0 → strong positive SHAP")


### 2.2 SHAP для XGBoost

Повторим анализ для XGBoost (обычно более точная модель).

In [ ]:
print("=" * 70)
print("SHAP ANALYSIS: XGBOOST")
print("=" * 70)

# TreeExplainer для XGBoost
print("\nCreating TreeExplainer for XGBoost...")
xgb_explainer = shap.TreeExplainer(models['XGBoost'])

# Вычисляем SHAP values
print("Computing SHAP values...")
xgb_shap_values = xgb_explainer.shap_values(X_test_scaled.iloc[:1000])

print(f"✅ SHAP values computed")
print(f"Shape: {xgb_shap_values.shape}")

# Summary plot
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Summary plot
plt.sca(axes[0])
shap.summary_plot(
    xgb_shap_values,
    X_test_scaled.iloc[:1000],
    feature_names=feature_cols,
    show=False
)
axes[0].set_title('SHAP Summary Plot: XGBoost', fontsize=14, fontweight='bold')

# Bar plot
plt.sca(axes[1])
shap.summary_plot(
    xgb_shap_values,
    X_test_scaled.iloc[:1000],
    feature_names=feature_cols,
    plot_type='bar',
    show=False
)
axes[1].set_title('SHAP Feature Importance: XGBoost', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Feature importance comparison
print("\n🎯 Топ-10 признаков по SHAP (XGBoost):")
xgb_mean_abs_shap = np.abs(xgb_shap_values).mean(axis=0)
xgb_feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Mean |SHAP|': xgb_mean_abs_shap
}).sort_values('Mean |SHAP|', ascending=False)

print(xgb_feature_importance.head(10).to_string(index=False))


#### 2.2.1 Сравнение Random Forest vs XGBoost

Сравним feature importance для обеих моделей.

In [ ]:
# Сравниваем feature importance RF vs XGBoost
comparison = pd.DataFrame({
    'Feature': feature_cols,
    'RF SHAP': mean_abs_shap,
    'XGB SHAP': xgb_mean_abs_shap
}).sort_values('XGB SHAP', ascending=False)

# Визуализация
fig, ax = plt.subplots(figsize=(12, 8))

x = np.arange(len(comparison.head(12)))
width = 0.35

ax.barh(x - width/2, comparison.head(12)['RF SHAP'], width, label='Random Forest', alpha=0.8)
ax.barh(x + width/2, comparison.head(12)['XGB SHAP'], width, label='XGBoost', alpha=0.8)

ax.set_yticks(x)
ax.set_yticklabels(comparison.head(12)['Feature'])
ax.set_xlabel('Mean |SHAP value|', fontsize=12)
ax.set_title('Feature Importance Comparison: RF vs XGBoost', fontsize=14, fontweight='bold')
ax.legend()
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Наблюдения:")
print("- Обе модели согласны в топовых признаках (capital-gain, age, education-num)")
print("- Ranking может немного отличаться")
print("- XGBoost может выявлять более тонкие interactions")
print("\n→ Консенсус между моделями повышает уверенность в интерпретации!")


---

## 🔬 Часть 3: LIME Analysis

### Что такое LIME?

**LIME (Local Interpretable Model-agnostic Explanations)** - метод для объяснения **отдельных predictions** любой модели.

**Как работает:**
1. Берём интересующий нас sample
2. Генерируем perturbed samples (слегка изменяем признаки)
3. Получаем predictions для них от черного ящика
4. Обучаем **простую интерпретируемую модель** (linear/decision tree) локально
5. Интерпретируем простую модель

**LIME vs SHAP:**

| Аспект | LIME | SHAP |
|--------|------|------|
| Теоретическое обоснование | ❌ Heuristic | ✅ Game theory (Shapley) |
| Consistency | ❌ Не гарантируется | ✅ Гарантируется |
| Скорость (local) | ✅ Быстрее | ⚠️ KernelSHAP медленнее |
| Скорость (tree models) | ⚠️ Медленнее | ✅ TreeSHAP очень быстро |
| Stability | ❌ Может варьироваться | ✅ Стабильнее |
| Global interpretation | ❌ Нет | ✅ Да |
| Use case | Quick local explanations | Production, требуется consistency |

**Когда использовать LIME:**
- ✅ Быстрые ad-hoc локальные объяснения
- ✅ Модели, для которых нет TreeSHAP (custom black boxes)
- ✅ Text/Image classification (LIME хорошо работает с ними)
- ⚠️ НЕ для production, где важна consistency

---


In [ ]:
print("=" * 70)
print("LIME ANALYSIS")
print("=" * 70)

# Создаём LIME explainer
print("\nCreating LIME Tabular Explainer...")
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train_scaled.values,
    feature_names=feature_cols,
    class_names=['<=50K', '>50K'],
    mode='classification',
    random_state=42
)

print("✅ LIME explainer created")

# Объясняем тот же high income sample, что и с SHAP
high_income_sample = X_test_scaled.iloc[high_income_idx].values

print(f"\nExplaining sample {high_income_idx} (predicted >50K)...")
lime_exp = lime_explainer.explain_instance(
    high_income_sample,
    models['Random Forest'].predict_proba,
    num_features=10,
    num_samples=1000  # сколько perturbed samples генерировать
)

print("✅ LIME explanation computed")


In [ ]:
# Визуализация LIME explanation
fig = lime_exp.as_pyplot_figure(label=1)  # label=1 для класса >50K
plt.title(f'LIME Explanation: Sample {high_income_idx} (>50K)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Текстовое представление
print("\n📊 LIME Explanation (Top features):")
print("="*50)
for feature, weight in lime_exp.as_list(label=1)[:10]:
    print(f"{feature:40s} : {weight:+.4f}")

print("\n💡 Интерпретация:")
print("- Positive weight → increases P(>50K)")
print("- Negative weight → decreases P(>50K)")
print("- Величина weight = strength of effect")


#### 3.1 Сравнение LIME vs SHAP

Сравним объяснения для того же sample.

In [ ]:
# Сравнение LIME vs SHAP для одного sample
print("=" * 70)
print(f"COMPARISON: LIME vs SHAP (Sample {high_income_idx})")
print("=" * 70)

# LIME feature importances
lime_dict = dict(lime_exp.as_list(label=1))
lime_values = {}
for feature in feature_cols:
    # LIME возвращает строки типа "feature <= value"
    for lime_feature, value in lime_dict.items():
        if feature in lime_feature:
            lime_values[feature] = value
            break
    if feature not in lime_values:
        lime_values[feature] = 0.0

# SHAP values для этого sample
shap_dict = dict(zip(feature_cols, rf_shap_values_class1[high_income_idx]))

# Создаём сравнительный DataFrame
comparison_local = pd.DataFrame({
    'Feature': feature_cols,
    'LIME': [lime_values.get(f, 0) for f in feature_cols],
    'SHAP': [shap_dict[f] for f in feature_cols]
})

# Сортируем по абсолютному значению SHAP
comparison_local['abs_SHAP'] = comparison_local['SHAP'].abs()
comparison_local = comparison_local.sort_values('abs_SHAP', ascending=False).head(10)

print("\nTop 10 features:")
print(comparison_local[['Feature', 'LIME', 'SHAP']].to_string(index=False))

# Визуализация
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(comparison_local))
width = 0.35

ax.barh(x - width/2, comparison_local['LIME'], width, label='LIME', alpha=0.8)
ax.barh(x + width/2, comparison_local['SHAP'], width, label='SHAP', alpha=0.8)

ax.set_yticks(x)
ax.set_yticklabels(comparison_local['Feature'])
ax.set_xlabel('Feature Importance (local explanation)', fontsize=12)
ax.set_title(f'LIME vs SHAP: Sample {high_income_idx}', fontsize=14, fontweight='bold')
ax.legend()
ax.invert_yaxis()
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Наблюдения:")
print("- LIME и SHAP обычно согласны в топовых признаках")
print("- Magnitude может отличаться (разные шкалы)")
print("- SHAP более стабильный и теоретически обоснованный")
print("- LIME быстрее для quick ad-hoc explanations")


---

## 📊 Часть 4: Partial Dependence Plots (PDP)

### Что такое PDP?

**Partial Dependence Plot** показывает **глобальное влияние** признака на predictions:
- Как меняется среднее prediction при изменении признака?
- Независимо от других признаков (marginalized)

**Как работает:**
1. Фиксируем интересующий признак на разных значениях
2. Для каждого значения: делаем predictions для всего датасета
3. Усредняем predictions
4. Строим график: feature value vs average prediction

**Преимущества:**
- ✅ Global view (не только локальное объяснение)
- ✅ Визуально понятно
- ✅ Показывает нелинейные эффекты

**Ограничения:**
- ⚠️ **Assumes independence:** если признаки коррелируют, может вводить в заблуждение
- ⚠️ **Averages out heterogeneity:** не показывает индивидуальные вариации

**ICE (Individual Conditional Expectation) curves:**
- Решает проблему averaging
- Показывает зависимость для каждого sample отдельно
- PDP = average of ICE curves

---


In [ ]:
print("=" * 70)
print("PARTIAL DEPENDENCE PLOTS (PDP)")
print("=" * 70)

# Топ признаки для PDP
pdp_features = ['age', 'education-num', 'hours-per-week', 'capital-gain']

print(f"\nBuilding PDP for: {pdp_features}")

# PDP для Random Forest
fig, ax = plt.subplots(figsize=(14, 8))

display = PartialDependenceDisplay.from_estimator(
    models['Random Forest'],
    X_test_scaled.iloc[:500],  # используем подвыборку для скорости
    features=pdp_features,
    feature_names=feature_cols,
    ax=ax,
    n_jobs=-1
)

plt.suptitle('Partial Dependence Plots: Random Forest', fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("\n✅ PDP completed")
print("\n📊 Интерпретация:")
print("- age: С возрастом вероятность >50K растёт (до определенного момента)")
print("- education-num: Больше образования → выше вероятность >50K")
print("- hours-per-week: Больше часов работы → выше income")
print("- capital-gain: Наличие capital gain → резко увеличивает вероятность")


In [ ]:
# 2D PDP для взаимодействий
print("\n2D Partial Dependence: Feature Interactions")
print("="*50)

fig, ax = plt.subplots(figsize=(10, 8))

# PDP для взаимодействия age и education-num
display_2d = PartialDependenceDisplay.from_estimator(
    models['Random Forest'],
    X_test_scaled.iloc[:500],
    features=[('age', 'education-num')],
    feature_names=feature_cols,
    ax=ax,
    n_jobs=-1
)

plt.suptitle('2D PDP: Age × Education Interaction', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Interaction effect:")
print("- Светлые области = высокая вероятность >50K")
print("- Тёмные области = низкая вероятность >50K")
print("- Видим joint effect возраста и образования")


### 4.1 Permutation Importance

**Permutation Importance** - простой и эффективный метод для оценки важности признаков:

**Алгоритм:**
1. Вычисляем baseline metric (accuracy, AUC, etc.) на test set
2. Для каждого признака:
   - Shuffle значения признака (breaks relationship с target)
   - Вычисляем metric заново
   - Importance = baseline - shuffled metric
3. Repeat несколько раз для стабильности

**Преимущества:**
- ✅ **Model-agnostic** (работает с любой моделью)
- ✅ **Handles correlations** (в отличие от Gini importance)
- ✅ **Intuitive interpretation**

**vs Gini Importance (Feature Importance в Random Forest):**
- Gini importance: сколько признак используется в splits
- **Проблема:** bias к high-cardinality features
- Permutation Importance: реальное влияние на predictions

---


In [ ]:
print("=" * 70)
print("PERMUTATION IMPORTANCE")
print("=" * 70)

# Вычисляем permutation importance для Random Forest
print("\nComputing Permutation Importance (это может занять минуту)...")
perm_importance = permutation_importance(
    models['Random Forest'],
    X_test_scaled.iloc[:1000],
    y_test.iloc[:1000],
    n_repeats=10,
    random_state=42,
    n_jobs=-1,
    scoring='roc_auc'
)

print("✅ Permutation Importance computed")

# Сортируем признаки по importance
perm_sorted_idx = perm_importance.importances_mean.argsort()[::-1]

# Создаём DataFrame
perm_df = pd.DataFrame({
    'Feature': np.array(feature_cols)[perm_sorted_idx],
    'Importance': perm_importance.importances_mean[perm_sorted_idx],
    'Std': perm_importance.importances_std[perm_sorted_idx]
})

print("\n📊 Top 10 Features by Permutation Importance:")
print(perm_df.head(10).to_string(index=False))


In [ ]:
# Визуализация Permutation Importance
fig, ax = plt.subplots(figsize=(10, 8))

top_n = 15
ax.barh(
    range(top_n),
    perm_df.head(top_n)['Importance'],
    xerr=perm_df.head(top_n)['Std'],
    alpha=0.8
)

ax.set_yticks(range(top_n))
ax.set_yticklabels(perm_df.head(top_n)['Feature'])
ax.set_xlabel('Importance (decrease in ROC AUC)', fontsize=12)
ax.set_title('Permutation Importance: Random Forest', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Интерпретация:")
print("- Высокая importance → shuffling признака сильно ухудшает модель")
print("- Error bars показывают variance (стабильность)")
print("- Отрицательная importance → признак не помогает (шум)")


In [ ]:
# Сравнение Gini vs Permutation Importance
print("\n" + "=" * 70)
print("COMPARISON: Gini Importance vs Permutation Importance")
print("=" * 70)

# Gini importance (встроенная в Random Forest)
gini_importance = models['Random Forest'].feature_importances_

# Создаём сравнительный DataFrame
importance_comparison = pd.DataFrame({
    'Feature': feature_cols,
    'Gini': gini_importance,
    'Permutation': perm_importance.importances_mean
}).sort_values('Permutation', ascending=False)

# Визуализация
fig, ax = plt.subplots(figsize=(12, 8))

x = np.arange(12)
width = 0.35

ax.barh(x - width/2, importance_comparison.head(12)['Gini'], width, label='Gini (MDI)', alpha=0.8)
ax.barh(x + width/2, importance_comparison.head(12)['Permutation'], width, label='Permutation', alpha=0.8)

ax.set_yticks(x)
ax.set_yticklabels(importance_comparison.head(12)['Feature'])
ax.set_xlabel('Importance', fontsize=12)
ax.set_title('Gini vs Permutation Importance', fontsize=14, fontweight='bold')
ax.legend()
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Наблюдения:")
print("- Gini и Permutation обычно согласны в топовых признаках")
print("- Gini может переоценивать high-cardinality features")
print("- Permutation более надёжный для feature selection")
print("- SHAP > Permutation > Gini (в плане теоретической обоснованности)")


---

## ⚖️ Часть 5: Fairness Analysis

### Почему Fairness важен?

**Проблема:** Высокая accuracy ≠ справедливая модель

**Real-world примеры bias:**
- **Amazon Recruiting AI (2018):** модель дискриминировала женщин (обучалась на исторических данных с bias)
- **COMPAS (Criminal Justice):** алгоритм предсказания recidivism показывал racial bias
- **Credit Scoring:** модели могут дискриминировать по возрасту, полу, расе
- **Healthcare:** алгоритмы могут хуже работать для minority groups

**Законодательные требования:**
- 🇺🇸 **Equal Credit Opportunity Act** - запрещает дискриминацию в кредитовании
- 🇪🇺 **GDPR Article 22** - право на объяснение автоматизированных решений
- 🇺🇸 **Fair Housing Act** - fairness в жилищном кредитовании

### Типы Fairness Metrics:

#### 1. **Demographic Parity (Statistical Parity)**
```
P(Y_pred = 1 | Sex = Male) = P(Y_pred = 1 | Sex = Female)
```
- Одинаковая доля positive predictions в каждой группе
- **Проблема:** игнорирует базовые различия в ground truth

#### 2. **Equal Opportunity**
```
P(Y_pred = 1 | Y_true = 1, Sex = Male) = P(Y_pred = 1 | Y_true = 1, Sex = Female)
```
- Одинаковый True Positive Rate для всех групп
- **Лучше:** учитывает ground truth

#### 3. **Equalized Odds**
```
Equal Opportunity + Equal False Positive Rate
```
- И TPR, и FPR одинаковые для всех групп

#### 4. **Calibration**
```
P(Y_true = 1 | Y_pred_proba = p, Sex = Male) = P(Y_true = 1 | Y_pred_proba = p, Sex = Female)
```
- Predicted probabilities соответствуют реальным для всех групп

**Trade-offs:**
- ⚠️ **Невозможно** удовлетворить все fairness criteria одновременно (теорема impossibility)
- ⚠️ **Fairness vs Accuracy trade-off** - иногда нужно жертвовать accuracy

---


In [ ]:
print("=" * 70)
print("FAIRNESS ANALYSIS")
print("=" * 70)

# Анализируем fairness по полу (sensitive attribute)
print("\nAnalyzing fairness by SEX...")

# Predictions для test set
rf_predictions = results['Random Forest']['predictions']
rf_probabilities = results['Random Forest']['probabilities']

# Добавляем predictions к sensitive attributes
fairness_df = sensitive_test.copy()
fairness_df['y_true'] = y_test.values
fairness_df['y_pred'] = rf_predictions
fairness_df['y_prob'] = rf_probabilities

print("\n" + "=" * 70)
print("1. DEMOGRAPHIC PARITY (Statistical Parity)")
print("=" * 70)

# Demographic Parity: доля positive predictions по группам
demo_parity = fairness_df.groupby('sex')['y_pred'].mean()
print("\nP(Y_pred = 1 | Sex):")
for sex, prob in demo_parity.items():
    print(f"  {sex:10s}: {prob:.4f} ({prob*100:.2f}%)")

# Disparate Impact Ratio
female_rate = demo_parity.get('Female', demo_parity.iloc[0])
male_rate = demo_parity.get('Male', demo_parity.iloc[1])
disparate_impact = female_rate / male_rate if male_rate > 0 else 0

print(f"\nDisparate Impact Ratio: {disparate_impact:.4f}")
print("  (Female rate / Male rate)")
print("  ✅ Good if between 0.8 and 1.25 (80% rule)")
if 0.8 <= disparate_impact <= 1.25:
    print("  ✅ PASS: Demographic Parity satisfied")
else:
    print(f"  ⚠️ FAIL: Potential bias (ratio = {disparate_impact:.2f})")


In [ ]:
print("\n" + "=" * 70)
print("2. EQUAL OPPORTUNITY (True Positive Rate)")
print("=" * 70)

# Equal Opportunity: TPR по группам (среди тех, кто действительно >50K)
positive_class = fairness_df[fairness_df['y_true'] == 1]
tpr_by_sex = positive_class.groupby('sex')['y_pred'].mean()

print("\nTrue Positive Rate (Recall) by Sex:")
for sex, tpr in tpr_by_sex.items():
    print(f"  {sex:10s}: {tpr:.4f} ({tpr*100:.2f}%)")

# Difference in TPR
tpr_diff = abs(tpr_by_sex.iloc[0] - tpr_by_sex.iloc[1])
print(f"\nDifference in TPR: {tpr_diff:.4f}")
if tpr_diff < 0.05:
    print("  ✅ PASS: Equal Opportunity satisfied (diff < 5%)")
else:
    print(f"  ⚠️ WARNING: TPR differs by {tpr_diff*100:.1f}%")


In [ ]:
print("\n" + "=" * 70)
print("3. FALSE POSITIVE RATE (FPR) BY SEX")
print("=" * 70)

# FPR по группам (среди тех, кто действительно <=50K)
negative_class = fairness_df[fairness_df['y_true'] == 0]
fpr_by_sex = negative_class.groupby('sex')['y_pred'].mean()

print("\nFalse Positive Rate by Sex:")
for sex, fpr in fpr_by_sex.items():
    print(f"  {sex:10s}: {fpr:.4f} ({fpr*100:.2f}%)")

# Difference in FPR
fpr_diff = abs(fpr_by_sex.iloc[0] - fpr_by_sex.iloc[1])
print(f"\nDifference in FPR: {fpr_diff:.4f}")
if fpr_diff < 0.05:
    print("  ✅ PASS: FPR similar across groups (diff < 5%)")
else:
    print(f"  ⚠️ WARNING: FPR differs by {fpr_diff*100:.1f}%")


In [ ]:
# Визуализация fairness metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Demographic Parity
demo_parity.plot(kind='bar', ax=axes[0, 0], color=['#e74c3c', '#3498db'])
axes[0, 0].set_title('Demographic Parity: P(Y_pred=1 | Sex)', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Positive Prediction Rate')
axes[0, 0].set_xlabel('Sex')
axes[0, 0].tick_params(axis='x', rotation=0)
axes[0, 0].axhline(y=demo_parity.mean(), color='green', linestyle='--', label='Overall mean')
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. True Positive Rate
tpr_by_sex.plot(kind='bar', ax=axes[0, 1], color=['#e74c3c', '#3498db'])
axes[0, 1].set_title('Equal Opportunity: TPR by Sex', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('True Positive Rate')
axes[0, 1].set_xlabel('Sex')
axes[0, 1].tick_params(axis='x', rotation=0)
axes[0, 1].axhline(y=tpr_by_sex.mean(), color='green', linestyle='--', label='Overall mean')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. False Positive Rate
fpr_by_sex.plot(kind='bar', ax=axes[1, 0], color=['#e74c3c', '#3498db'])
axes[1, 0].set_title('FPR by Sex', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('False Positive Rate')
axes[1, 0].set_xlabel('Sex')
axes[1, 0].tick_params(axis='x', rotation=0)
axes[1, 0].axhline(y=fpr_by_sex.mean(), color='green', linestyle='--', label='Overall mean')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Confusion Matrix by Sex
from sklearn.metrics import confusion_matrix

# Aggregate metrics для визуализации
metrics_by_sex = {}
for sex in fairness_df['sex'].unique():
    sex_data = fairness_df[fairness_df['sex'] == sex]
    metrics_by_sex[sex] = {
        'Accuracy': accuracy_score(sex_data['y_true'], sex_data['y_pred']),
        'Precision': precision_score(sex_data['y_true'], sex_data['y_pred']),
        'Recall': recall_score(sex_data['y_true'], sex_data['y_pred']),
        'F1': f1_score(sex_data['y_true'], sex_data['y_pred'])
    }

metrics_df = pd.DataFrame(metrics_by_sex).T
metrics_df.plot(kind='bar', ax=axes[1, 1], width=0.8)
axes[1, 1].set_title('Performance Metrics by Sex', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Score')
axes[1, 1].set_xlabel('Sex')
axes[1, 1].tick_params(axis='x', rotation=0)
axes[1, 1].legend(loc='lower right')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.suptitle('Fairness Analysis: Sex-based Metrics', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n📊 Summary Table:")
print(metrics_df.round(4))


In [ ]:
# Calibration Analysis
print("\n" + "=" * 70)
print("4. CALIBRATION ANALYSIS")
print("=" * 70)

# Calibration curves по группам
from sklearn.calibration import calibration_curve

fig, ax = plt.subplots(figsize=(10, 8))

for sex in fairness_df['sex'].unique():
    sex_data = fairness_df[fairness_df['sex'] == sex]
    
    fraction_of_positives, mean_predicted_value = calibration_curve(
        sex_data['y_true'],
        sex_data['y_prob'],
        n_bins=10,
        strategy='uniform'
    )
    
    ax.plot(
        mean_predicted_value,
        fraction_of_positives,
        marker='o',
        linewidth=2,
        label=sex
    )

# Perfect calibration line
ax.plot([0, 1], [0, 1], 'k--', label='Perfect Calibration')

ax.set_xlabel('Mean Predicted Probability', fontsize=12)
ax.set_ylabel('Fraction of Positives', fontsize=12)
ax.set_title('Calibration Curve by Sex', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Интерпретация Calibration:")
print("- Линия близка к diagonal → модель well-calibrated")
print("- Above diagonal → модель underestimates probabilities")
print("- Below diagonal → модель overestimates probabilities")
print("- Если кривые для разных групп различаются → calibration bias")


### 5.1 Bias Mitigation Strategies

Если обнаружен bias, что делать?

#### 1. **Pre-processing** (до обучения модели)
- **Reweighting:** давать больший вес underrepresented groups
- **Resampling:** oversample minority, undersample majority
- **Data augmentation:** генерировать синтетические примеры
- **Feature removal:** удалить sensitive attributes (но осторожно - может не помочь из-за proxy features)

#### 2. **In-processing** (во время обучения)
- **Fairness constraints:** добавить fairness penalty в loss function
- **Adversarial debiasing:** обучить adversarial network, чтобы predictions не зависели от sensitive attribute
- **Fair representations:** learn embedding, где sensitive info удалена

#### 3. **Post-processing** (после обучения)
- **Threshold optimization:** разные thresholds для разных групп
- **Calibration:** recalibrate probabilities per group
- **Reject option:** не делать prediction, если confidence низкая

**Trade-offs:**
- ⚠️ **Fairness vs Accuracy:** mitigation может снизить overall accuracy
- ⚠️ **Which fairness metric?** невозможно удовлетворить все одновременно
- ⚠️ **Legal considerations:** некоторые методы (e.g., different thresholds) могут быть illegal

**Best practices:**
1. ✅ **Measure fairness** на test set для всех sensitive groups
2. ✅ **Document** bias analysis и mitigation efforts (model cards)
3. ✅ **Monitor** fairness in production (drift)
4. ✅ **Stakeholder involvement:** involve affected communities
5. ✅ **Transparency:** объяснять decisions, особенно для high-stakes applications

---


---

## 🌳 Часть 6: Decision Tree Visualization

### Model-Specific Interpretability

**Decision Trees** - inherently interpretable модели:
- Каждое решение - это simple if-then rule
- Путь от root до leaf объясняет prediction
- Можно визуализировать весь process

**Преимущества:**
- ✅ **Complete transparency:** видим все правила
- ✅ **Easy to explain:** можно объяснить любому
- ✅ **No preprocessing needed:** работает с categorical features

**Trade-off:**
- ⚠️ **Accuracy vs Interpretability:**
  - Shallow tree: interpretable, но менее точная
  - Deep tree: точная, но сложная для интерпретации
  - Random Forest / XGBoost: ещё точнее, но black box (нужен SHAP)

**Когда использовать:**
- ✅ **High-stakes decisions:** медицина, кредиты (нужна полная прозрачность)
- ✅ **Regulatory compliance:** когда требуется объяснение каждого правила
- ✅ **Knowledge extraction:** понять domain logic из данных

---


In [ ]:
print("=" * 70)
print("DECISION TREE VISUALIZATION")
print("=" * 70)

# Обучаем простую Decision Tree для интерпретации
print("\nTraining interpretable Decision Tree (max_depth=4)...")
dt = DecisionTreeClassifier(
    max_depth=4,  # ограничиваем глубину для interpretability
    min_samples_split=100,
    min_samples_leaf=50,
    random_state=42
)

dt.fit(X_train_scaled, y_train)
dt_pred = dt.predict(X_test_scaled)

# Метрики
dt_accuracy = accuracy_score(y_test, dt_pred)
dt_f1 = f1_score(y_test, dt_pred)
dt_auc = roc_auc_score(y_test, dt.predict_proba(X_test_scaled)[:, 1])

print(f"\n✅ Decision Tree trained")
print(f"Accuracy: {dt_accuracy:.4f}")
print(f"F1-Score: {dt_f1:.4f}")
print(f"ROC AUC: {dt_auc:.4f}")
print(f"\nТочность ниже, чем у XGBoost/RF, но дерево полностью interpretable!")


In [ ]:
# Визуализация Decision Tree
fig, ax = plt.subplots(figsize=(25, 15))

plot_tree(
    dt,
    feature_names=feature_cols,
    class_names=['<=50K', '>50K'],
    filled=True,
    rounded=True,
    ax=ax,
    fontsize=10,
    proportion=True  # показываем пропорции вместо абсолютных чисел
)

plt.title('Decision Tree Visualization (max_depth=4)', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n📊 Как читать дерево:")
print("- Каждый узел: условие split (e.g., capital-gain <= 0.5)")
print("- gini: impurity (0 = pure, 0.5 = 50/50)")
print("- samples: доля samples в этом узле")
print("- value: [proportion class 0, proportion class 1]")
print("- class: predicted class (цвет показывает majority class)")
print("\nОранжевый = <=50K, Синий = >50K")


In [ ]:
# Текстовое представление правил
print("\n" + "=" * 70)
print("DECISION RULES (TEXT FORMAT)")
print("=" * 70)

tree_rules = export_text(dt, feature_names=feature_cols)
print(tree_rules)

print("\n💡 Примеры правил:")
print("- IF capital-gain > 0.5 AND ... → Predicted: >50K")
print("- IF age <= 0.3 AND education-num <= 0.2 → Predicted: <=50K")
print("\nЭти правила можно показать бизнесу / регуляторам!")


In [ ]:
# Feature Importance из Decision Tree
dt_importance = dt.feature_importances_

importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': dt_importance
}).sort_values('Importance', ascending=False)

# Визуализация
fig, ax = plt.subplots(figsize=(10, 6))

top_n = 12
ax.barh(
    range(top_n),
    importance_df.head(top_n)['Importance'],
    alpha=0.8,
    color='steelblue'
)

ax.set_yticks(range(top_n))
ax.set_yticklabels(importance_df.head(top_n)['Feature'])
ax.set_xlabel('Feature Importance (Gini)', fontsize=12)
ax.set_title('Decision Tree Feature Importance', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 Топ-5 признаков в Decision Tree:")
print(importance_df.head(5).to_string(index=False))


---

## 📋 Часть 7: Comprehensive Summary & Best Practices

### Сравнение всех XAI методов

| Метод | Scope | Speed | Theory | Use Case |
|-------|-------|-------|--------|----------|
| **SHAP** | Global + Local | ⚠️ Medium (TreeSHAP fast) | ✅ Game theory | Production, требуется consistency |
| **LIME** | Local only | ✅ Fast | ❌ Heuristic | Quick ad-hoc explanations |
| **PDP** | Global | ✅ Fast | ✅ Solid | Global feature effects |
| **Permutation Importance** | Global | ⚠️ Medium | ✅ Solid | True feature importance |
| **Gini Importance** | Global | ✅ Very fast | ⚠️ Biased | Quick check (tree models) |
| **Decision Tree Viz** | Global | ✅ Fast | ✅ Complete | Full transparency |
| **Attention Weights** | Local | ✅ Fast | ⚠️ Debatable | Transformers only |

---

### 🎯 Рекомендации по выбору метода

#### 1. **Production ML System (High Stakes)**
- ✅ **Primary:** SHAP (теоретически обоснован, consistent)
- ✅ **Secondary:** Permutation Importance (global view)
- ✅ **Fairness:** Demographic Parity, Equal Opportunity
- ✅ **Documentation:** Model cards, fairness reports

**Пример:** Credit scoring, healthcare diagnostics, hiring

#### 2. **Quick Prototyping / Research**
- ✅ **LIME** (fast local explanations)
- ✅ **PDP** (global trends)
- ✅ **Feature Importance** (quick check)

**Пример:** Kaggle competition, exploratory analysis

#### 3. **Regulatory Compliance (Full Transparency)**
- ✅ **Decision Tree** visualization
- ✅ **Rule extraction**
- ✅ **Linear models** с coefficient interpretation
- ⚠️ Возможно придётся жертвовать accuracy

**Пример:** Regulated industries (banking, insurance), government

#### 4. **Deep Learning / Transformers**
- ✅ **DeepSHAP** или **GradientSHAP**
- ✅ **Attention visualization** (с осторожностью)
- ✅ **Integrated Gradients**
- ✅ **Saliency Maps** (для images/text)

**Пример:** NLP, computer vision, time series with Transformers

---

### ⚖️ Fairness Best Practices

1. **Identify Sensitive Attributes:**
   - Gender, race, age, disability status, etc.
   - Legal requirements vary by jurisdiction

2. **Choose Fairness Metric(s):**
   - **Demographic Parity:** равные positive rates
   - **Equal Opportunity:** равные TPR (часто preferred)
   - **Calibration:** equal calibration по группам
   - ⚠️ **Невозможно** удовлетворить все одновременно!

3. **Measure Fairness:**
   - На **test set** (не на train!)
   - Для **всех** sensitive groups
   - **Before** и **after** mitigation

4. **Bias Mitigation:**
   - **Pre-processing:** reweighting, resampling
   - **In-processing:** fairness constraints in loss
   - **Post-processing:** threshold optimization

5. **Documentation:**
   - **Model Cards** (Google best practice)
   - Fairness metrics in production dashboard
   - Regular audits (quarterly/yearly)

6. **Continuous Monitoring:**
   - Fairness может drift со временем
   - Data distribution changes → fairness changes
   - Monitor в production!

---

### 🚀 Production Deployment Checklist

**Before deploying ML model:**

- [ ] **Interpretability:**
  - [ ] SHAP values computed для test set
  - [ ] Global feature importance documented
  - [ ] Local explanation API available
  - [ ] PDP plots generated для топовых features

- [ ] **Fairness:**
  - [ ] Sensitive attributes identified
  - [ ] Fairness metrics measured (Demographic Parity, Equal Opportunity)
  - [ ] Bias mitigation applied (if needed)
  - [ ] Fairness report документирован

- [ ] **Documentation:**
  - [ ] Model card created
  - [ ] Training data описана (source, biases, limitations)
  - [ ] Evaluation metrics по всем groups
  - [ ] Known limitations документированы
  - [ ] Ethical considerations discussed

- [ ] **Monitoring:**
  - [ ] Dashboard для monitoring fairness в production
  - [ ] Alerts на drift в fairness metrics
  - [ ] Logging для SHAP values (sample)

- [ ] **Legal/Regulatory:**
  - [ ] Legal review completed
  - [ ] GDPR compliance (право на объяснение)
  - [ ] Industry-specific regulations checked

---


### 📚 Key Takeaways

#### 1. **Interpretability is NOT Optional**
- Регулируемые индустрии требуют объяснений (GDPR, FDA, etc.)
- Stakeholders хотят понимать, как работает модель
- Debugging: interpretability помогает найти bugs и bias

#### 2. **SHAP - Gold Standard**
- Теоретически обоснован (Shapley values)
- Consistent, local + global
- TreeSHAP очень fast для tree-based моделей
- **Use SHAP в production**

#### 3. **Different Methods, Different Insights**
- **SHAP:** feature attribution (сколько каждый признак вносит)
- **PDP:** global trends (как признак влияет в среднем)
- **LIME:** quick local explanations
- **Permutation:** true importance (handles correlations)
- **Все вместе** дают comprehensive understanding!

#### 4. **Fairness ≠ Equal Outcomes**
- Fairness - это про equal treatment, не equal outcomes
- Выбор fairness metric зависит от use case и законодательства
- Trade-off: fairness vs accuracy (иногда неизбежен)
- **Measure, mitigate, monitor**

#### 5. **Accuracy vs Interpretability Trade-off**
- Simple models (linear, shallow trees): interpretable, но менее точные
- Complex models (deep learning, ensembles): точные, но black box
- **XAI methods** (SHAP, LIME) позволяют использовать complex models с объяснениями
- **Best of both worlds!**

#### 6. **Documentation & Transparency**
- Model cards - best practice от Google
- Документировать training data, biases, limitations
- Transparency builds trust
- Регуляторы требуют документацию

#### 7. **Continuous Monitoring**
- Fairness и interpretability не one-time tasks
- Data drift → fairness drift
- Monitor в production, регулярные audits

---

### 🔗 Связь с другими фазами

**Phase 6 использует знания из:**
- **Phase 1-2:** Базовые модели (Logistic Regression, Neural Networks)
- **Phase 3:** Ensemble methods (Random Forest, XGBoost) → TreeSHAP
- **Phase 4:** Transformers → Attention visualization
- **Phase 5:** Anomaly detection → interpretable outliers

**Phase 6 готовит к:**
- **Phase 7:** Production & MLOps → monitoring fairness, model cards

---

### 🎓 Real-World Impact

**XAI спасает проекты:**
- **Amazon (2018):** Hiring AI дискриминировала женщин → проект закрыт
  - ✅ **Lesson:** Measure fairness before deployment!
- **COMPAS:** Racial bias в criminal justice
  - ✅ **Lesson:** Audit third-party models!
- **Healthcare:** FDA требует объяснения AI diagnoses
  - ✅ **Lesson:** Interpretability = regulatory requirement

**XAI создаёт ценность:**
- **Кредитный скоринг:** Объяснение отказа → клиент знает, как улучшить
- **Медицина:** Врач понимает AI диагноз → больше доверия
- **Fraud detection:** Аналитик видит, почему транзакция suspicious → быстрее реагирует
- **Бизнес insights:** SHAP показывает, какие факторы влияют на churn → action plan

---

## 🎉 Phase 6 Complete!

**Вы теперь знаете:**
- ✅ SHAP (TreeSHAP, KernelSHAP, DeepSHAP)
- ✅ LIME
- ✅ Partial Dependence Plots (PDP)
- ✅ Permutation Importance
- ✅ Fairness metrics (Demographic Parity, Equal Opportunity, Calibration)
- ✅ Bias mitigation strategies
- ✅ Decision Tree visualization
- ✅ Production best practices

**Следующий шаг: Phase 7 - Production & MLOps**
- Model deployment (FastAPI, Docker, Cloud)
- Monitoring (data drift, model drift, fairness drift)
- CI/CD для ML
- Experiment tracking (MLflow, Weights & Biases)
- Версионирование (DVC)

---

**🚀 Ready for production-ready ML!**
